In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style("darkgrid")

In [ ]:
df=pd.read_csv("perrin-freres-monthly-champagne-.csv")

In [ ]:
df.columns=["Month", "Sales"]
df.head()

In [ ]:
df.tail()

In [ ]:
df=df.iloc[:-2,:]
df.tail()

In [ ]:
df.info()

In [ ]:
df["Month"]=pd.to_datetime(df["Month"])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#df=df.set_index("Month")

In [ ]:
#df.head()

In [ ]:
df.describe()

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
plt.figure(figsize=(8,4))
sns.lineplot(x="Month", y="Sales", data=df)

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(df["Sales"])

In [ ]:
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(series):
    results=adfuller(series)
    labels=["ADF test stats", "p-value", "# Lags Used", "# Observation used"]
    for result, label in zip(results, labels):
        print(f"{result} : {label}")
        
    if(results[1]<=0.05):
        print("Null Hypothesis rejected. Data is Stationary")
    else:
        print("Null Hypothesis stands. Data is non-stationary")

In [ ]:
adfuller_test(df["Sales"])

In [ ]:
df.head()

In [ ]:
df["Seasonal First Diff"]=df["Sales"]-df["Sales"].shift(12)

In [ ]:
df.head(14)

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(x="Month", y="Seasonal First Diff", data=df)

In [ ]:
adfuller_test(df["Seasonal First Diff"].dropna())

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig=plt.figure(figsize=(10,6))
ax1=fig.add_subplot(211)
fig=plot_acf(df["Seasonal First Diff"].dropna(), lags=40, ax=ax1)
ax2=fig.add_subplot(212)
fig=plot_pacf(df["Seasonal First Diff"].dropna(), lags=40, ax=ax2)

p=1, q=0 or 1

We did seasonal differencing once, so d=1

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model=ARIMA(df["Sales"], order=(1,1,1))
model_fit=model.fit()

In [ ]:
df["Forecast"]=model_fit.predict(start=90, end=103, dynamic=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(x="Month", y="Sales", data=df)
sns.lineplot(x="Month", y="Forecast", data=df)

In [ ]:
import statsmodels.api as sm

In [ ]:
model=sm.tsa.statespace.SARIMAX(df['Sales'],order=(1, 1, 1),seasonal_order=(1,1,1,12))
results=model.fit()

In [ ]:
df["Forecast"]=results.predict(start=90,end=103,dynamic=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(x="Month", y="Sales", data=df)
sns.lineplot(x="Month", y="Forecast", data=df)

In [ ]:
from pandas.tseries.offsets import DateOffset

In [ ]:
future_dates=[df.iloc[-1,0]+ DateOffset(months=x) for x in range(1,24)]

In [ ]:
future_dates

In [ ]:
future_dates_df=pd.DataFrame(future_dates)
future_dates_df.columns=["Month"]
future_dates_df["Sales"]=np.nan
future_dates_df.head()

In [ ]:
future_df=pd.concat([df,future_dates_df])
future_df=future_df.reset_index(drop=True)

In [ ]:
future_df.head()

In [ ]:
future_df.tail()

In [ ]:
len(df)

In [ ]:
len(future_df)

In [ ]:
future_df.iloc[[103,104,105,106],:]

In [ ]:
future_df["Sales"]=results.predict(start=104, end=129, dynamic=True)

In [ ]:
future_df.iloc[[103,104,105,106],:]

In [ ]:
future_df.tail()

In [ ]:
forecast_df=future_df.iloc[104:129,:]

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(x="Month", y="Sales", data=df)
sns.lineplot(x="Month", y="Sales", data=forecast_df, color="red")